In [21]:
import os
import mne
import shutil

import numpy as np
import scipy.io as io

from typing import List

In [26]:
filename = "dane_eeg.zip"
extract_dir = "dane_eeg"
seizure_occ_file = "dane_eeg/czasy_napadow.m"
healthy_dir = "dane_eeg/zdrowi"
unhealthy_dir = "dane_eeg/chorzy"

# shutil.unpack_archive(filename, extract_dir)

In [27]:

def get_mat_files(base_path): 
    mat_files_list = []
    filenames = {}

    for file in os.listdir(base_path):
        if file.endswith(".mat"):
            mat_files_list.append(base_path + "/" + file)
            
    for filename in mat_files_list:        
        filenames[filename.split('/')[-1].split('_')[0]] = filename
        
    return mat_files_list, filenames


healthy_paths, healthy_patients_dict = get_mat_files(healthy_dir)
unhealthy_paths, unhealthy_patients_dict = get_mat_files(unhealthy_dir)

In [29]:
import re

def get_all_seizure_occurrences(file_path):
    all_occurrences = {}
    with open (file_path , 'r') as file:
        for line in file :
            line_number = int(re.search(r'{([0-9]+)}', line).group(1))
            occ_list = list(map(int, re.search(r'\[(.+?)]', line).group(1).split()))
            all_occurrences[line_number] = occ_list
    return all_occurrences


seizure_occurences = get_all_seizure_occurrences(seizure_occ_file)

In [30]:
class Seizure:
    def __init__ (self, id, filename, samples, seizure_occurences=None, frequency=100, channels=16) -> None :
        super (). __init__ ()
        self.id = id
        self.filename = filename
        self.samples = samples
        self.seizure_occurences = seizure_occurences
        self.frequency = frequency
        self.channels = channels

In [34]:
def healthy_patients(healthy_people_dict):
    healthy_patients = []
    for id, path in healthy_people_dict.items():
        samples = io.loadmat(path)['x']
        filename = os.path.basename(path)
        healthy_patients.append(Seizure(id, filename, samples))
 
    return healthy_patients

def unhealthy_patients(unhealthy_people_dict, seizure_occurences):
    unhealthy_patients = []
    for id, path in unhealthy_people_dict.items():
        seizure_occ = seizure_occurences[int(id)]
        samples = io.loadmat(path)['x']
        filename = os.path.basename(path)
        unhealthy_patients.append(Seizure(id, filename, samples, seizure_occurences=seizure_occ))
 
    return unhealthy_patients

In [35]:
healthy_list: List[Seizure] = healthy_patients(healthy_patients_dict)
unhealthy_list: List[Seizure] = unhealthy_patients(unhealthy_patients_dict, seizure_occurences)

In [40]:
def saveToPng(figure, filename, seizure_time):
    base_filename = os.path.splitext(filename)[0]
    dir_name = dir_img + base_filename
    if not os.path.exists(dir_name):
        os. makedirs( dir_name )
    figure.savefig(dir_name + '/' + base_filename + str(seizure_time) + '.png')


def plot_all_attacks(unhealthy_people, save_to_png=False):
 # colored blocks for visualization of attack
    def create_annotations( matches ):
        events = [x for x in matches ]
        return mne.Annotations(onset=events, duration=2, description=['seizure ' + str(x) for x in matches])

    def visualize(filename, samples, seizure_time, all_events, save_to_png):
        ch_names = list(map(str,range(1,17))) 
        info = mne.create_info(ch_names=ch_names, sfreq =100)
        raw = mne.io.RawArray(np.transpose(samples), info)
        raw.set_annotations(all_events)

        figure = raw.plot(n_channels=ch_number, scalings='auto', title=filename + 'Time of seizure: ' + str(seizure_time), block=False, start=max(0, seizure_time - 5), duration=10)
 
        if save_to_png:
            saveToPng(figure, filename, seizure_time)

    for person in unhealthy_people:
        all_events = create_annotations(person.seizure_occurences)
        for seizure_time in person.seizure_occurences:
            visualize(person.filename, person.samples, seizure_time, all_events, save_to_png)

# use only once for creating images of seizures
plot_all_attacks(unhealthy_list, save_to_png = True)